# CryoCloud Tutorials: SlideRule

Tutorial Leads: [Scott Henderson](mailto:scottyh@uw.edu) and [Tyler Sutterley](mailto:tsutterl@uw.edu)

Learning Objectives
-------------------
- Learn how to use SlideRule APIs
- Learn about using ICESat-2 data in coordination with other datasets
- Learn about interactive maps

### SlideRule Introduction

SlideRule is an on-demand science data processing service that runs in on Amazon Web Services and responds to REST API calls to process and return science results.  SlideRule was designed to enable researchers and other data systems to have low-latency access to custom-generated, high-level, analysis-ready data products using processing parameters supplied at the time of the request. 

The SlideRule ICESat-2 plug-in has a cloud-optimized version of the [land ice algorithm (ATL06)](https://nsidc.org/data/atl06) that can process the lower-level [geolocated photon height data products (ATL03)](https://nsidc.org/data/atl03) hosted on AWS by the NSIDC DAAC.  This work supports science applications for the NASA Ice Cloud and land Elevation Satellite-2 (ICESat-2) mission. 

[Documentation for using SlideRule](https://slideruleearth.io/web/rtd/) is available from the [project website](https://slideruleearth.io/web/) 

#### **Q: What does SlideRule ICESat-2 actually do?**
SlideRule creates a simplified version of the [ICESat-2 land ice height product](https://nsidc.org/data/atl06) that can be adjusted to suit different needs.  SlideRule let's you create customized ICESat-2 segment heights _directly_ from the photon height data anywhere on the globe, _on-demand_ and quickly.

In [ ]:
import io
import logging, warnings
import geopandas as gpd
import sliderule.icesat2
import sliderule.io
import sliderule.ipysliderule
import matplotlib.pyplot as plt
import shapely.geometry
import owslib.wms

# autoreload
%load_ext autoreload
%autoreload 2

#### Initiate SlideRule API
- Sets the URL for accessing the SlideRule service
- Builds a table of servers available for processing data

In [ ]:
# set the url for the sliderule service
# set the logging level
sliderule.icesat2.init("slideruleearth.io", loglevel=logging.WARNING)
# turn off warnings for tutorial
warnings.filterwarnings('ignore')

### Set options for making science data processing requests to SlideRule

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets = sliderule.ipysliderule.widgets()
SRwidgets.set_atl06_defaults()
SRwidgets.VBox(SRwidgets.atl06(display='basic'))

### Interactive Mapping with Leaflet

Interactive maps within the SlideRule python API are build upon [ipyleaflet](https://ipyleaflet.readthedocs.io), which are Jupyter and python bindings for the fantastic [Leaflet](https://leafletjs.com/) javascript library.

### Select regions of interest for submitting to SlideRule

Here, we create polygons or bounding boxes for our regions of interest.  
This map is also our viewer for inspecting our SlideRule ICESat-2 data returns.

In [ ]:
# create ipyleaflet map in specified projection
m = sliderule.ipysliderule.leaflet(SRwidgets.projection.value,
    center=(37.8, -122.4), zoom=11,
    full_screen_control=True)
# install click handler callback
m.add_selected_callback(SRwidgets.atl06_click_handler)
m.map

### Build and transmit requests to SlideRule

- SlideRule will query the [NASA Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/) for ATL03 data within our region of interest
- When using the `icesat2` asset, the ICESat-2 ATL03 data are then accessed from the NSIDC AWS s3 bucket in `us-west-2`
- The ATL03 granules is spatially subset within SlideRule to our exact region of interest
- SlideRule then uses our specified parameters to calculate average height segments from the ATL03 data in parallel
- The completed data is streamed concurrently back and combined into a geopandas GeoDataFrame within the Python client

In [ ]:
%%time
# build sliderule parameters using latest values from widget
parms = SRwidgets.build_atl06()

# clear existing geodataframe results
elevations = [sliderule.emptyframe()]

# for each region of interest
for poly in m.regions:
    # add polygon from map to sliderule parameters
    parms["poly"] = poly
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    elevations.append(sliderule.icesat2.atl06p(parms))
# concatenate the results into a single geodataframe
gdf = gpd.pd.concat(elevations)

### Review GeoDataFrame output
Can inspect the columns, number of returns and returns at the top of the GeoDataFrame.

See the [ICESat-2 documentation](https://slideruleearth.io/rtd/user_guide/ICESat-2.html#elevations) for descriptions of each column

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

### Add GeoDataFrame to map

For stability of the leaflet map, SlideRule will as a default limit the plot to have up to 10000 points from the GeoDataFrame

GeoDataFrames can be plotted in any available [matplotlib colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
SRwidgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
])

In [ ]:
%matplotlib inline
# ATL06-SR fields for hover tooltip
fields = m.default_atl06_fields()
m.GeoData(gdf, column_name=SRwidgets.column_name, cmap=SRwidgets.colormap,
    max_plot_points=10000, tooltip=True, colorbar=True, fields=fields)

### Create plots for a single track
- `cycles`: Will plot all available cycles of data returned by SlideRule for a single RGT and ground track
- `scatter`: Will plot data returned by SlideRule for a single RGT, ground track and cycle

To select a track from the leaflet plot above, click on one of the plotted elevations and the RGT and Cycle will automatically get populated below

The cycles plots should only be used in regions with [repeat Reference Ground Track (RGT) pointing](https://icesat-2.gsfc.nasa.gov/science/specs)

In [ ]:
SRwidgets.VBox([
    SRwidgets.plot_kind,
    SRwidgets.rgt,
    SRwidgets.ground_track,
    SRwidgets.cycle,
])

In [ ]:
%matplotlib widget
# default is to skip cycles with significant off-pointing
SRwidgets.plot(gdf, kind=SRwidgets.plot_kind.value, cycle_start=3,
    legend=True, legend_frameon=False)

### Advanced ICESat-2 SlideRule Example

SlideRule also can use different sources for photon classification before calculating the average segment height.  
This is useful for example, in cases where there may be a vegetated canopy affecting the spread of the photon returns.
- ATL03 photon confidence values, based on algorithm-specific classification types for land, ocean, sea-ice, land-ice, or inland water
- [ATL08 Land and Vegetation Height product](https://nsidc.org/data/atl08) photon classification
- Experimental [YAPC (Yet Another Photon Classification)](https://github.com/tsutterley/yapc) photon-density-based classification

#### Leaflet Basemaps and Layers

There are 3 projections available within SlideRule for mapping ([Global](https://epsg.io/3857), [North](https://epsg.io/5936) and [South](https://epsg.io/3031)).  There are also contextual layers available for each projection.

<table>
  <tbody>
    <tr>
      <th align='center' max-width="30%"><a href="https://epsg.io/3857">Global (Web Mercator, EPSG:3857)</a></th>
      <th align='center' max-width="30%"><a href="https://epsg.io/5936">North (Alaska Polar Stereographic, EPSG:5936)</a></th>
      <th align='center' max-width="30%"><a href="https://epsg.io/3031">South (Antarctic Polar Stereographic, EPSG:3031)</a></th>
    </tr>
    <tr>
      <td align='left' valign='top' width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="https://apps.nationalmap.gov/3depdem/">USGS 3DEP Hillshade</a></li>
            <li><a href="https://asterweb.jpl.nasa.gov/gdem.asp">ASTER GDEM Hillshade</a></li>
            <li><a href="https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9">ESRI Imagery</a></li>
            <li><a href="http://glims.colorado.edu/glacierdata/">Global Land Ice Measurements from Space (GLIMS)</a></li>
            <li><a href="https://www.glims.org/RGI/">Randolph Glacier Inventory (RGI)</a></li>
        </ul>
       </td>
       <td align='left' valign='top' width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="http://goto.arcgisonline.com/maps/Arctic_Imagery">ESRI Imagery</a></li>
            <li><a href="https://www.pgc.umn.edu/data/arcticdem">ArcticDEM</a></li>
        </ul>
       </td>
       <td align='left' valign='top' width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="https://lima.usgs.gov/">Landsat Image Mosaic of Antarctica (LIMA)</a></li>
            <li><a href="https://nsidc.org/data/nsidc-0280">MODIS Mosaic of Antarctica (MOA)</a></li>
            <li><a href="https://nsidc.org/data/NSIDC-0103">Radarsat Antarctic Mapping Project (RAMP)</a></li>
            <li><a href="https://www.pgc.umn.edu/data/rema">Reference Elevation Model of Antarctica (REMA)</a></li>
        </ul>
      </td>
    </tr>
  </tbody>
</table>

In addition, most [xyzservice providers](https://xyzservices.readthedocs.io/en/stable/introduction.html) can be added as contextual layers to the global Web Mercator maps

In [ ]:
SRwidgets.projection.value = 'North'
SRwidgets.layers.value = ['ArcticDEM']
SRwidgets.VBox([
    SRwidgets.projection,
    SRwidgets.layers,
    SRwidgets.raster_functions
])

In [ ]:
# create ipyleaflet map in specified projection
m1 = sliderule.ipysliderule.leaflet(SRwidgets.projection.value)
# install click handler callback
m1.add_selected_callback(SRwidgets.atl06_click_handler)
m1.map

In [ ]:
m1.add_layer(
    layers=SRwidgets.layers.value,
    rendering_rule=SRwidgets.rendering_rule
)
# read and add region of interest
reg = gpd.read_file('Fenris_Gletscher_PS_v1.4.2.geojson')
m1.add_region(sliderule.io.from_geodataframe(reg))

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets.classification.value = ['atl03']
SRwidgets.surface_type.value = 'Land ice'
SRwidgets.VBox(SRwidgets.atl06())

In [ ]:
%%time
# build sliderule parameters using latest values from widget
parms = SRwidgets.build_atl06()
# reduce data to a single cycle
parms["cycle"] = 19
# additionally sample ArcticDEM mosaic at ATL06-SR points
parms["samples"] = dict(mosaic= {
    "asset": "arcticdem-mosaic",
    "radius": 10.0,
    "zonal_stats": True
})

# clear existing geodataframe results
elevations = [sliderule.emptyframe()]

# for each region of interest
for poly in m1.regions:
    # add polygon from map to sliderule parameters
    parms["poly"] = poly
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    elevations.append(sliderule.icesat2.atl06p(parms))
# concatenate the results into a single geodataframe
gdf1 = gpd.pd.concat(elevations)

In [ ]:
%matplotlib inline
# ATL06-SR fields for hover tooltip
fields = m1.default_atl06_fields()
m1.GeoData(gdf1, column_name=SRwidgets.column_name, cmap=SRwidgets.colormap,
    max_plot_points=10000, tooltip=True, colorbar=True, fields=fields)

#### Create a static map with SlideRule returns
[geopandas GeoDataFrames can be transformed to different Coordinate Reference Systems (CRS)](http://geopandas.org/projections.html) using the `to_crs()` function.

Here, we can make a static map of Greenland containing our SlideRule returns.

In [ ]:
def plot_arcticdem(ax=None, **kwargs):
    """Plot ArcticDEM layer as a basemap

    Parameters
    ----------
    ax: obj, default None
        Figure axis
    kwargs: dict, default {}
        Additional keyword arguments for ``wms.getmap``
    """
    # set default keyword arguments
    kwargs.setdefault('layers', '0')
    kwargs.setdefault('format', 'image/png')
    kwargs.setdefault('srs', 'EPSG:3413')
    # create figure axis if non-existent
    if (ax is None):
        _, ax = plt.subplots()
    # get the pixel bounds and resolution of the map
    width = int(ax.get_window_extent().width)
    height = int(ax.get_window_extent().height)
    # calculate the size of the map in pixels
    kwargs.setdefault('size', [width, height])
    # calculate the bounding box of the map in projected coordinates
    bbox = [None]*4
    bbox[0], bbox[2] = ax.get_xbound()
    bbox[1], bbox[3] = ax.get_ybound()
    kwargs.setdefault('bbox', bbox)
    # url of ArcticDEM WMS
    url = ('http://elevation2.arcgis.com/arcgis/services/Polar/'
        'ArcticDEM/ImageServer/WMSserver')
    wms = owslib.wms.WebMapService(url=url, version='1.1.1')
    basemap = wms.getmap(**kwargs)
    # read WMS layer and plot
    img = plt.imread(io.BytesIO(basemap.read()))
    ax.imshow(img, extent=[bbox[0],bbox[2],bbox[1],bbox[3]])

In [ ]:
%matplotlib inline
# create polar stereographic plot of ATL06-SR data
crs = 'EPSG:3413'
# create figure axis
fig,ax1 = plt.subplots(num=1, ncols=3, sharex=True, sharey=True, figsize=(13,3.5))
ax1[0].set_aspect('equal', adjustable='box')

# calculate normalization for height plots
vmin, vmax = gdf1['h_mean'].quantile((0.02, 0.98)).values

# calculate difference between sliderule and ArcticDEM mosaic heights
gdf1['difference'] = gdf1['h_mean'] - gdf1['mosaic.mean']
filtered = gdf1[gdf1['difference'].abs() < 100].to_crs(crs)

# plot ATL06-SR elevation
label = f'SlideRule Elevation [m]'
sc = filtered.plot(ax=ax1[0], zorder=1, markersize=0.5,
    column='h_mean', cmap='viridis', vmin=vmin, vmax=vmax,
    legend=True, legend_kwds=dict(label=label, shrink=0.95, extend='both'))
sc.set_rasterized(True)

# plot ArcticDEM mosaic elevation
label = f'ArcticDEM Mosaic Elevation [m]'
sc = filtered.plot(ax=ax1[1], zorder=1, markersize=0.5,
    column='mosaic.mean', cmap='viridis', vmin=vmin, vmax=vmax,
    legend=True, legend_kwds=dict(label=label, shrink=0.95, extend='both'))
sc.set_rasterized(True)

# plot difference between heights
label = f'Elevation Differences [m]'
sc = filtered.plot(ax=ax1[2], zorder=1, markersize=0.5,
    column='difference', cmap='coolwarm', vmin=-40, vmax=40,
    legend=True, legend_kwds=dict(label=label, shrink=0.95, extend='both'))
sc.set_rasterized(True)

# convert regions into a geoseries object
regions = []
for poly in m1.regions:
    lon,lat = sliderule.io.from_region(poly)
    regions.append(shapely.geometry.Polygon(zip(lon,lat)))
gs = gpd.GeoSeries(regions, crs='EPSG:4326').to_crs(crs)

# add background and labels
for ax in ax1:
    # plot each region of interest
    gs.plot(ax=ax, facecolor='none', edgecolor='black', lw=3)
    # plot ArcticDEM as basemap
    plot_arcticdem(ax, srs=crs)
    # add x label
    ax.set_xlabel('{0} [{1}]'.format('Easting','m'))
# add y label
ax1[0].set_ylabel('{0} [{1}]'.format('Northing','m'))

# adjust subplot and show
fig.subplots_adjust(left=0.06, right=0.98, bottom=0.08, top=0.98, wspace=0.1)